In [14]:
import sys, os
import time
import argparse
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join('..') + '/scripts/sig_gen/')) # import sig gen folder
from sg_smb100a_generate_frequency_sweep_notebook_1 import SG_SOCK # Import the Signal Generator Socket class 

#-----------------------import libraries for Spectrum analyzer----------------#
sys.path.insert(0, os.path.abspath(os.path.join('..') + '/scripts/spec_ana/ms2090a/'))
from sa_ms2090a_setup_basic_params import SA_SOCK        # Import the Spectrum Analyser Socket class

#--------Import scpi database for Signal Generator and Spectrum analyzer ----------#
sys.path.insert(0, os.path.abspath(os.path.join('..') + '/resources/'))
from scpi_database import SGCmds
from scpi_database import SACmds

# -----------------Connection Settings----------------------
SG_PORT = 5025                    # default SMB R&S port 
SG_HOST = 'za-itf-signal-generator.ad.skatelescope.org' #'10.20.7.1'             # smb100a signal generator IP
SG_ADDRESS = (SG_HOST, SG_PORT)
SA_HOST = 'za-itf-spectrum-analyser.ad.skatelescope.org' #'10.20.7.4'             # ms2090a spectrum analyzer IP temporary
SA_PORT = 9001                    # Anritsu spectrum analyzer port 
SA_ADDRESS = (SA_HOST, SA_PORT)
#-----------------------------------------------------------
# ----------------Initialization of Variables---------------    
DEFAULT_TIMEOUT = 1        # Default socket timeout
RF_OFF = 0
RF_ON = 1

SHORT_DELAY = 0.1
LONG_DELAY = 1

#-----------------SA initialization Variables----------
NUMPOINTS = 631     # Number of measurement points (Max=625)
RBW = 3e6           # Resolution BW of spectrum analyser
VBW = 3e6           # Video BW of spectrum analyser

# SuT spec
sut_spec = -6 # 6 dB isol for 2-way Splitter in this case

In [15]:
#-------------------SG_SMB100A Setup-------------------------#
def setupSG():  
    print('/------Setup signal generator Class---------/\n')
    SG = SG_SOCK()                                 # Call main class
    print(f'Connected to: {SG.connectSG(SG_ADDRESS)}')    
    SG.setSGCmd(SGCmds['rf_state'], RF_ON)
    SG.setSGCmd(SGCmds['power'], -25) 
    print('/------End of Setup Signal Generator Setup---------/\n\n')
    return SG

In [16]:
# Set up parameters to be parsed 
start_freq = 0.009e9 # the start frequency incl. units (Hz)')
stop_freq = 4.2e9    # the stop frequency incl. units (Hz)')
step_freq = 6e6  # the step frequency incl. units (Hz)')
dwel_time = 0.8    # the sweep dwell time (ms)')


In [17]:
#------------------------SA_MS2090A Setup-------------------------#
def setupSA():
    print('/------Setup spectrum analyser---------/')
    SA = SA_SOCK()
    print(f'Connected to: {SA.connectSA(SA_ADDRESS)}')
    SA.setSACmd(SACmds['reset_device'])
    SA.setSACmd(SACmds['start_freq'], start_freq)
    SA.setSACmd(SACmds['stop_freq'], stop_freq)
    SA.setSACmd(SACmds['sweep_points'], NUMPOINTS)
    SA.setSACmd(SACmds['rbw_auto'], 'off')
    SA.setSACmd(SACmds['vbw_auto'], 'off')
    SA.setSACmd(SACmds['att_level'], 10)
    SA.setSACmd(SACmds['ref_level'], -10)
    SA.setSACmd(SACmds['det_mode'], 'RMS') 
    SA.setSACmd(SACmds['trace1_mode'], 'MAX')
    #SA.setSACmd(SACmds['marker1-state'], 'ON')
    print('/------End of Setup Spectrum Analyzer Setup---------/\n\n')
    return SA

In [18]:
#------------------------------ PLOT ---------------------------#
def plotTrace(freq_values, power_values, ylim_min, ylim_max): 
    ''' Plot response

    This function plots power vs frequency maxhold 

    @params:    
        freq_values: integer list [in Hz]
        power_value: integer list [in dBm]
        ref_level: integer [in dBm]
    '''
    x_axis = freq_values
    y_axis = power_values
    plt.figure(figsize=(10,10))
    plt.plot(x_axis, y_axis)
    plt.xlabel('Frequency in GHz', fontsize = 16)
    plt.ylabel('Power in dBm', fontsize = 16)
    plt.title('Sweep Maxhold Plot')
    plt.xlim(freq_values[0], freq_values[-1])
    plt.ylim(-110, ref_level)
    plt.show()


In [19]:
print('\n/--------- Running main Code ---------/') 

sg = setupSG()
time.sleep(1) 
sa = setupSA()
time.sleep(1) 


/--------- Running main Code ---------/
/------Setup signal generator Class---------/

Connected to: Rohde&Schwarz,SMB100A,1406.6000k03/183286,4.20.028.58
/------End of Setup Signal Generator Setup---------/


/------Setup spectrum analyser---------/
Connected to: Anritsu,MS2090A,2234010,V2022.9.1
/------End of Setup Spectrum Analyzer Setup---------/




In [20]:
'''
The following block of code generates a sweep frequency of the signal generator
at 100MHz step
@params:
        start_freq      : start frequency in Hz (not less than 9 kHz)
        stop_freq       : stop frequency in Hz (not more than 6 GHz)
        step_freq       : step frequency in Hz (default = 100 MHz)
        dwel_time       : duration of frequency output in ms (default=100 ms)
''' 
# Set and Display SG start frequency
sg.setSGCmd(SGCmds['start_freq'], start_freq)
start_freq_recvd = sg.getSGCmd(SGCmds['start_freq']).decode()
print(f'Signal Generator Start Frequency = {float(start_freq_recvd) / 1e6} MHz') 

# Set and Display SG stop frequency
sg.setSGCmd(SGCmds['stop_freq'], stop_freq)
stop_freq_recvd = int(sg.getSGCmd(SGCmds['stop_freq']).decode())
print(f'Signal Generator Stop Frequency = {float(stop_freq_recvd) / 1e9} GHz')

centFreq = (int(float(start_freq)) + int(float(stop_freq))) / 2
span = int(float(stop_freq)) - int(float(start_freq))    
span_recvd = float(sg.getSGCmd(SGCmds['span_freq']).decode())
print(f"Sweep Span = {span_recvd / 1e9} GHz")

# 1. Set the sweep range
sg.setSGCmd(SGCmds['cent_freq'], centFreq)
centFreq_recvd = float(sg.getSGCmd(SGCmds['cent_freq']).decode())
print(f"Sweep Center Frequency = {centFreq_recvd / 1e9} GHz")
sg.setSGCmd(SGCmds['span_freq'], span)

# 2. Select linear or logarithmic spacing
sg.setSGCmd(SGCmds['sweep_freq_spac_conf'], 'LIN')

# 3. Set the step width and dwell time
sg.setSGCmd(SGCmds['sweep_freq_step'], step_freq)
step_freq_recvd = int(float(sg.getSGCmd(SGCmds['step_freq']).decode()))
print(f"Step Frequency = {step_freq_recvd / 1e9} GHz")
sg.setSGCmd(SGCmds['sweep_freq_dwell'], dwel_time)

# 4. Select the trigger mode
sg.setSGCmd(SGCmds['sweep_freq_trig'], 'SING')

# 5. Select sweep mode and activate the sweep
sg.setSGCmd(SGCmds['sweep_freq_mode'], 'AUTO')
sg.setSGCmd(SGCmds['freq_mode'], 'SWE')

# -------------- Display Spectrum Analyzer Settings
# Read and print Spectrum Analyzer start frequency
start_freq = sa.getSACmd(SACmds['start_freq']).decode()
# print(f'Spectrum Analyzer Start Frequency = {float(start_freq) / 1e6} MHz') 
print(f'Spectrum Analyzer Start Frequency = {start_freq} Hz') 

# Read and print SA stop frequency
stop_freq = int(sa.getSACmd(SACmds['stop_freq']).decode())
print(f'Spectrum Analyzer Stop Frequency = {float(stop_freq) / 1e9} GHz')
print(f"Spectrum Analyzer Attenuation set to {sa.getSACmd(SACmds['att_level']).decode()} dB")
print(f"Spectrum Analyzer Reference Level set to {sa.getSACmd(SACmds['ref_level']).decode()} dBm")
# ------------ End of Display Spectrum Analyzer Settings

# 6. Trigger the sweep     
sg.setSGCmd(SGCmds['sweep_freq_exec'])
print('Executing sweep...')
# ----------------End of Signal Generator Setup Sweep Parameters -----------------------

# Wait until the sweep is finished
run_time_delay = int((float(stop_freq) - float(start_freq)) * (float(dwel_time) / float(step_freq)))
for count in range(0, run_time_delay, 10):
    time.sleep(10)          # wait for sweep to complete  
    # print(f'count = {count}...')
    print('Sweeping...')

Signal Generator Start Frequency = 9.0 MHz
Signal Generator Stop Frequency = 4.2 GHz
Sweep Span = 4.191 GHz
Sweep Center Frequency = 2.1045 GHz
Step Frequency = 0.001 GHz
Spectrum Analyzer Start Frequency = 9000000 Hz
Spectrum Analyzer Stop Frequency = 4.2 GHz
Spectrum Analyzer Attenuation set to 10 dB
Spectrum Analyzer Reference Level set to -10 dBm
Executing sweep...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Sweeping...
Swee

In [21]:
''' Read trace data
        
This block of code reads the power trace data and calculates the trace frequency points

'''         
    
freq_values = []
power_values = [] 

# Confugure trace data format to be Ascii
sa.setSACmd(SACmds['data_format'], 'ASC')

# Set single sweep
sa.setSACmd((SACmds['sing_sweep_state']), 'OFF')

# --------------- Get trace data power values ------------------  
# sa.getSACmd((SACmds['trace_data']), 1)     # Command not working, replaced by the chunk of code below
sa.sendall(bytes('TRACE:DATA? 1\n', encoding = 'utf8'))
time.sleep(SHORT_DELAY)
try:
    return_str = sa.recv(1024)
except sa.timeout:
    raise StopIteration('No data received from instrument') 
# -------------------- End of trace data power values reading -------


# --------------- Trace data power values conditioning -------------------
power_data = return_str
power_data = power_data.decode()
power_data = str(power_data)
power_data = power_data.split(',')              # Makes variable a list
power_data.pop(0)
print(power_data)
power_data = [float(x) for x in power_data]
    
No_of_Sweep_Points = int(sa.getSACmd(SACmds['sweep_points']).decode())   # Get No. of Sweep Points
power_values = power_data
# -------------------- End of trace data power values conditioning -------

freq_step_size = int((float(stop_freq) - float(start_freq)) / (No_of_Sweep_Points))
for i in range(0, (No_of_Sweep_Points - 1), 1):
    freq_values.append(int(float(start_freq)) + (i * freq_step_size))
    

print('Power and Frequency Values acquired...')

['-25.35', '-25.76', '-25.89', '-25.93', '-25.87', '-25.83', '-25.72', '-25.46', '-25.21', '-25.33', '-25.73', '-25.91', '-26', '-25.92', '-25.87', '-25.77', '-25.57', '-25.27', '-25.35', '-25.6', '-25.94', '-26.02', '-25.97', '-25.85', '-25.76', '-25.56', '-25.32', '-25.33', '-25.58', '-25.91', '-26', '-26.14', '-25.79', '-25.72', '-25.56', '-25.4', '-25.42', '-25.58', '-25.8', '-25.97', '-26.21', '-25.84', '-25.72', '-25.51', '-25.56', '-25.48', '-25.59', '-25.65', '-25.86', '-26.28', '-25.96', '-25.78', '-25.48', '-25.62', '-25.58', '-25.66', '-25.57', '-25.76', '-26.28', '-26.07', '-25.86', '-25.51', '-25.56', '-25.57', '-25.69', '-25.47', '-25.62', '-26.17', '-26.21', '-25.98', '-25.59', '-25.51', '-25.62', '-25.7', '-25.47', '-25.6', '-26.06', '-26.32', '-26.1', '-25.63', '-25.42', '-25.56', '-25.58', '-25.46', '-25.61', '-25.96', '-26.29', '-26.22', '-25.7', '-25.4', '-25.56', '-25.5', '-25.55', '-25.6', '-25.9', '-26.2', '-26.31', '-25.79', '-25.42', '-25.46', '-25.4', '-25.54'

ValueError: invalid literal for int() with base 10: '5.85,-26.1,-26.1,-26,-25.98,-25.89,-25.63,-25.35,-25.46,-25.75,-26.1,-26.14,-26.07,-25.94,-25.89,-25.67,-25.41,-25.43,-25.7,-26.01,-26.15,-26.21,-25.9,-25.84,-25.67,-25.49,-25.46,-25.63,-25.91,-26.11

In [ ]:
sa.setSACmd(SACmds['marker1_state'], 'ON')
sa.setSACmd(SACmds['marker2_state'], 'ON')
sa.setSACmd(SACmds['marker3_state'], 'ON')
sa.setSACmd(SACmds['marker4_state'], 'ON')
sa.setSACmd(SACmds['marker5_state'], 'ON')
sa.setSACmd(SACmds['marker6_state'], 'ON')
marker1_freq = float((stop_freq / 6) * 1)
marker2_freq = float((stop_freq / 6) * 2)
marker3_freq = float((stop_freq / 6) * 3)
marker4_freq = float((stop_freq / 6) * 4)
marker5_freq = float((stop_freq / 6) * 5)
marker6_freq = float((stop_freq / 6) * 6)
sa.setSACmd(SACmds['marker1_frequency'], int(marker1_freq))
sa.setSACmd(SACmds['marker2_frequency'], int(marker2_freq))
sa.setSACmd(SACmds['marker3_frequency'], int(marker3_freq))
sa.setSACmd(SACmds['marker4_frequency'], int(marker4_freq))
sa.setSACmd(SACmds['marker5_frequency'], int(marker5_freq))
sa.setSACmd(SACmds['marker6_frequency'], int(marker6_freq))
sa.setSACmd(SACmds['marker_table'], 'ON')
marker1_power = sa.getSACmd(SACmds['marker1_power']).decode()
marker2_power = sa.getSACmd(SACmds['marker2_power']).decode()
marker3_power = sa.getSACmd(SACmds['marker3_power']).decode()
marker4_power = sa.getSACmd(SACmds['marker4_power']).decode()
marker5_power = sa.getSACmd(SACmds['marker5_power']).decode()
marker6_power = sa.getSACmd(SACmds['marker6_power']).decode()
print(f'Marker 1 frequency = {marker1_freq / 1e9} GHz, Marker 1 Power = {marker1_power} dBm')
print(f'Marker 2 frequency = {marker2_freq / 1e9} GHz, Marker 2 Power = {marker2_power} dBm')
print(f'Marker 3 frequency = {marker3_freq / 1e9} GHz, Marker 3 Power = {marker3_power} dBm')
print(f'Marker 4 frequency = {marker4_freq / 1e9} GHz, Marker 4 Power = {marker4_power} dBm')
print(f'Marker 5 frequency = {marker5_freq / 1e9} GHz, Marker 5 Power = {marker5_power} dBm')
print(f'Marker 6 frequency = {marker6_freq / 1e9} GHz, Marker 6 Power = {marker6_power} dBm')

ref_level = int(sa.getSACmd(SACmds['ref_level']).decode())

if float(marker1_power) <= float(sut_spec + ref_level) \
and float(marker2_power) <= float(sut_spec + ref_level) \
and float(marker3_power) <= float(sut_spec + ref_level) \
and float(marker4_power) <= float(sut_spec + ref_level) \
and float(marker5_power) <= float(sut_spec + ref_level) \
and float(marker6_power) <= float(sut_spec + ref_level):
    print('Test Pass')
else:
    print('Test Fail')

In [ ]:
scale_per_div = int(sa.getSACmd(SACmds['scale_per_div']).decode())
ylim_min = ref_level
ylim_max = int(scale_per_div * ref_level)
print('Displayed plot...')

# Save data points as CSV
trace_data_points = []
i = 0
while i < No_of_Sweep_Points:                            
    file = open (r'./Maxhold_Sweep_Trace_Data_File.csv', 'w')         
    file.write("Frequency in GHz;Power in dBm\n")  
    x=0                                           
    while x < int((No_of_Sweep_Points-1)):                 
        file.write(str(freq_values[x]))    
        file.write(";")                            
        file.write(str(power_values[x]))           
        file.write("\n")                           
        x=x+1                                      
    file.close()

plotTrace(freq_values, power_values, ylim_min, ylim_max)

In [ ]:
sg.setSGCmd(SGCmds['rf_state'], RF_OFF)
sg.closeSGSock()
sa.closeSASock()
print('End of program.')